In [2]:
import statistics as stats
from tqdm import tqdm
import csv
import datetime as datet

epoch = datet.datetime.utcfromtimestamp(0)
def unix_time_secs(dt):
    return (dt - epoch).total_seconds()

def str_to_datetime(dt_str):
     return datet.datetime.strptime(dt_str, '%Y%m%d %H:%M:%S.%f')

data = "data/EURUSD_Tick_Data_Jan-Feb.csv"
outputCSV = "data/EURUSD_Tick_Data_Jan-Feb_POC.csv"    
window = 14400     # amount of time in seconds
startdt = unix_time_secs(str_to_datetime("20230101 22:00:00.000"))

with open(data, newline='') as csvfile:
    with open(outputCSV, mode="w", newline='') as csvwriter:
        Reader = csv.reader(csvfile, delimiter=',')
        Writer = csv.writer(csvwriter, delimiter=',')
        Writer.writerow(["Datetime", "Bid", "Ask", "AveragePrice", "PocPrice", "Volume"])
        next(Reader)

        windowData, pocPrice = [], None

        for row in tqdm(Reader):
            datetime = row[0]  # DATETIME OF EVERY ROW
            bidPrice = float(row[1])  # BID PRICE OF EVERY ROW
            askPrice = float(row[2])  # ASK PRICE OF EVERY ROW
            volume = float(row[5])  # VOLUME OF EVERY ROW
            averagePrice = (bidPrice + askPrice) / 2  # AVERAGE BID-ASK PRICE OF EVERY ROW
            
            dt = str_to_datetime(datetime) # convert str to datetime
                
            if unix_time_secs(dt) - startdt >= window:
                startdt = unix_time_secs(dt)        # reset starting datetime
                if len(windowData) > 0:
                    pocPrice = stats.mode(windowData) # find the POC for the window data
                    windowData.clear()                # clear the window data
            else:
                windowData.append(averagePrice)       # creating the first period's data
            

            Writer.writerow([datetime, bidPrice, askPrice, averagePrice, pocPrice, volume])

5658748it [01:28, 64056.87it/s]
